# Landsat Imagery - Remote Sensing Indexes

A Landsat image was used as an example to calculate different indexes such as: 

1. Normalized Difference Vegetation Index (NDVI)
2. Enhanced Vegetation Index (EVI)
3. Normalized Difference Water Index (NDWI)
4. Modified Normalized Difference Water Index (MNDWI)
5. Normalized Difference Built-Up Index (NDBI)
7. Soil Adjustment Vegetation Index (SAVI)

Use the configuration tool at the right-top to visualize the desired index and use the different options available to interact with the map, including the drawing, plotting and timelapse creation tools. 

In [1]:
import geemap
import ee
import os
import geemap.colormaps as cm

%reload_ext autoreload
%autoreload 2

#ee.Authenticate()
ee_token = os.environ.get("EARTHENGINE_TOKEN")
if ee_token is not None:
    credential_file_path = os.path.expanduser("~/.config/earthengine/")
    if not os.path.exists(credential_file_path):
        credential = '{"refresh_token":"%s"}' % ee_token
        os.makedirs(credential_file_path, exist_ok=True)
        with open(credential_file_path + "credentials", "w") as file:
            file.write(credential)
ee.Initialize()

In [2]:
# Example data : 
#'LANDSAT/LE7_TOA_5YEAR/1999_2003'
#NDVI('LANDSAT/LE7_TOA_5YEAR/1999_2003',7)

In [3]:
cm.palettes.dem
cm.palettes.ndvi
cm.palettes.ndwi

Map = geemap.Map()

palette = cm.palettes.ndvi
#palette = cm.palettes.terrain

ndvi = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
vis_params = {
  'min': -1,
  'max': 1,
  'palette': palette}

Map.add_colorbar(vis_params)
Map


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
#If it's not working with the indexes.py you can use this functions
#Be careful, here you just need 2 args. 

def index_NDVI(name, sat):
    """
    Calculate the NDVI (Normalized Difference Vegetation Index)
    @param name: path of the gee image - String 
           sat: Number of the Landsat used as source - number
    @return boolean False if the image is not coming from Ladsat 7 or 8
                    else, return True
    """
    image = ee.Image(name)
    if(sat == 7): 
        NIR = image.select('B4')
        Red = image.select('B3')
    elif(sat != 7 or sat != 8): 
        print("Error: Not a Landsat 7 or 8 image!")
        return False 
    elif(sat == 8): 
        NIR = image.select('B5')
        Red = image.select('B4')
    
    # Formula of NDVI = (NIR - Red)/(NIR + Red)
    ndvi = NIR.subtract(Red).divide(NIR.add(Red))

    palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']

    Map.addLayer(ndvi, {'palette': palette}, "NDVI")
    return True
    
def index_EVI(name, sat):
    """
    Calculate the EVI (Enhanced Vegetation Index)
    @param name: path of the gee image - String 
           sat: Number of the Landsat used as source - number
    @return boolean False if the image is not coming from Ladsat 7 or 8
                    else, return True
    """
    image = ee.Image(name)
    
    #Formula of EVI = 2.5 ((NIR - Red)/(NIR + 6*Red - 7.5*Blue + 1))
    if(sat == 7): 
        evi = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': image.select('B4'),
          'RED': image.select('B3'),
          'BLUE': image.select('B1')})
    elif(sat == 8): 
        evi = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': image.select('B5'),
          'RED': image.select('B4'),
          'BLUE': image.select('B2')})
    else: 
        print("Error: Not a Landsat 7 or 8 image!")
        return False
    
    Map.addLayer(evi, {'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']}, 'EVI')
    return True

def index_NDWI(name, sat): 
    """
    Calculate the NDWI (Normalized Difference Water Index)
    @param name: path of the gee image - String 
           sat: Number of the Landsat used as source - number
    @return boolean False if the image is not coming from Ladsat 7 or 8
                    else, return True
    """
    image = ee.Image(name)
    if(sat == 7): 
        NIR = image.select('B4')
        SWIR = image.select('B5')
    elif(sat == 8): 
        NIR = image.select('B5')
        SWIR = image.select('B6')
    else: 
        print("Error : Not a Landsat 7 or 8 image!")
        return False
    #Formula of NDWI = (NIR - SWIR)/(NIR + SWIR)
    ndwi = NIR.subtract(SWIR).divide(NIR.add(SWIR))
    
    Map.addLayer(ndwi, {'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']}, "NDWI")
    return True 

def index_MNDWI(name, sat): 
    """
    Calculate the MNDWI (Modified Normalized Difference Water Index)
    @param name: path of the gee image - String 
           sat: Number of the Landsat used as source - number
    @return boolean False if the image is not coming from Ladsat 7 or 8
                    else, return True
    """
    image = ee.Image(name)
    if(sat == 7): 
        Green = image.select('B2')
        SWIR = image.select('B5')
    elif(sat == 8): 
        Green = image.select('B3')
        SWIR = image.select('B6')
    else: 
        print("Error : Not a Landsat 7 or 8 image!")
        return False
    #Formula of MNDWI = (Green - SWIR)/(Green + SWIR)
    mndwi = Green.subtract(SWIR).divide(Green.add(SWIR))
    
    Map.addLayer(mndwi, {'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']}, "MNDWI")
    return True 

def index_NDBI(name, sat): 
    """
    Calculate the NDBI (Normalized Difference Built-up Index)
    @param name: path of the gee image - String 
           sat: Number of the Landsat used as source - number
    @return boolean False if the image is not coming from Ladsat 7 or 8
                    else, return True
    """
    image = ee.Image(name)
    if(sat == 7): 
        NIR = image.select('B4')
        SWIR = image.select('B5')
    elif(sat == 8): 
        NIR = image.select('B5')
        SWIR = image.select('B6')
    else: 
        print("Error : Not a Landsat 7 or 8 image!")
        return False
    
    #Formula of NDBI = (SWIR - NIR)/(SWIR + NIR)
    ndbi = SWIR.subtract(NIR).divide(SWIR.add(NIR))
    
    Map.addLayer(ndbi, {'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']}, "NDBI")
    return True

def index_SAVI(name, sat):
    """
    Calculate the SAVI (Soil Adjustment Vegetation Index)
    @param name: path of the gee image - String 
           sat: Number of the Landsat used as source - number
    @return boolean False if the image is not coming from Ladsat 7 or 8
                    else, return True    
    """
    image = ee.Image(name);
    if(sat == 7): 
        savi = image.expression('((NIR - Red)/(NIR + Red + 0.5))*1.5', {
          'NIR': image.select('B4'),
          'Red': image.select('B3')})
    elif(sat == 8): 
        savi = image.expression('((NIR - Red)/(NIR + Red + 0.5))*1.5', {
          'NIR': image.select('B5'),
          'Red': image.select('B4')})
    else: 
        print("Error : Not a Landsat 7 or 8 image!")
        return False
    
    Map.addLayer(savi, {'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']}, "SAVI")
    return True

index_SAVI('LANDSAT/LE7_TOA_5YEAR/1999_2003',7)
index_NDWI('LANDSAT/LE7_TOA_5YEAR/1999_2003',7)
index_NDBI('LANDSAT/LE7_TOA_5YEAR/1999_2003',7)
index_MNDWI('LANDSAT/LE7_TOA_5YEAR/1999_2003',7)
index_EVI('LANDSAT/LE7_TOA_5YEAR/1999_2003',7)
index_NDVI('LANDSAT/LE7_TOA_5YEAR/1999_2003',7)

True